# 0. Librerías

In [1]:
import pandas as pd
import glob, os

# 01. Carga de datos

In [2]:
base_path = 'C:\\Users\\monts\\Practica2\\project'

In [3]:
all_files = sorted(glob.glob(os.path.join(base_path, "data_origen", "*.csv")), reverse = True)
all_files

['C:\\Users\\monts\\Practica2\\project\\data_origen\\conjunto_de_datos_defunciones_registradas_2023.CSV',
 'C:\\Users\\monts\\Practica2\\project\\data_origen\\conjunto_de_datos_defunciones_registradas_2022.CSV',
 'C:\\Users\\monts\\Practica2\\project\\data_origen\\conjunto_de_datos_defunciones_registradas_2021.csv',
 'C:\\Users\\monts\\Practica2\\project\\data_origen\\conjunto_de_datos_defunciones_registradas_2020.csv']

In [4]:
dfs = []
for f in all_files:
    print("Leyendo", f)
    aux = pd.read_csv(f, low_memory=False)  # si pesan mucho, puedes usar chunks
    dfs.append(aux)

Leyendo C:\Users\monts\Practica2\project\data_origen\conjunto_de_datos_defunciones_registradas_2023.CSV
Leyendo C:\Users\monts\Practica2\project\data_origen\conjunto_de_datos_defunciones_registradas_2022.CSV
Leyendo C:\Users\monts\Practica2\project\data_origen\conjunto_de_datos_defunciones_registradas_2021.csv
Leyendo C:\Users\monts\Practica2\project\data_origen\conjunto_de_datos_defunciones_registradas_2020.csv


# 02. Revisión y homologación

In [5]:
# Revisión de tamaño y columnas de cada df
column_sets = {}

for i, df in enumerate(dfs):
    num_rows = len(df)
    cols = set(df.columns)
    
    # Nombre de columnas
    col_str = ", ".join(sorted(df.columns))
    column_sets[i] = col_str
    
    # Tamaño de df
    print(f"DF {i+1} (Registros: {num_rows}):")
    print(f"  Columnas ({len(cols)}): {col_str[:120]}...") # Imprimimos solo una parte
    print("-" * 30)

DF 1 (Registros: 799869):
  Columnas (74): afromex, anio_cert, anio_nacim, anio_ocur, anio_regis, area_ur, asist_medi, capitulo, causa_def, cirugia, cod_adicio, co...
------------------------------
DF 2 (Registros: 847716):
  Columnas (74): afromex, anio_cert, anio_nacim, anio_ocur, anio_regis, area_ur, asist_medi, capitulo, causa_def, cirugia, cod_adicio, co...
------------------------------
DF 3 (Registros: 1122249):
  Columnas (59): anio_cert, anio_nacim, anio_ocur, anio_regis, area_ur, asist_medi, capitulo, causa_def, complicaro, cond_act, cond_cert,...
------------------------------
DF 4 (Registros: 1086743):
  Columnas (59): anio_cert, anio_nacim, anio_ocur, anio_regis, area_ur, asist_medi, capitulo, causa_def, complicaro, cond_act, cond_cert,...
------------------------------


## 02.1. Homologación

In [6]:
# Se tomará como base 2020, que está en la posición 3 de los dfs
index = 3

dfs[index].columns = dfs[index].columns.str.lower().str.strip()
base_columns = dfs[index].columns.tolist()

In [7]:
# Conservar la variable de clasificación de la defunción como tipo_defun
if 'presunto' in base_columns:
     base_columns.remove('presunto')
if 'tipo_defun' not in base_columns:
     base_columns.append('tipo_defun')

In [8]:
# Construcción de df finales: agregando indicador de año, homologando nombre de variable y reduciendo a las variables base
final_dfs = []
for i, df in enumerate(dfs):
    # Identificador del año de cada db
    year = 2023 - i
    df['year_db'] = year 
    
    # Normalización de columnas
    df.columns = df.columns.str.lower().str.strip() # Nombres en minúsculas y sin espacios
    
    # Homologar 'presunto' a 'tipo_defun'
    if 'presunto' in df.columns:
        df = df.rename(columns={'presunto': 'tipo_defun'}, inplace=False)

    # Ajuste de columnas base, incluyendo el año
    if 'year_db' not in base_columns:
         base_columns.append('year_db')

    # Extracción y reordenamiento final
    df_reindexed = df.reindex(columns = base_columns)
    final_dfs.append(df_reindexed) # para aplicar la selección de columnas de la plantilla

In [9]:
# Consolidación de db
df_consolidado = pd.concat(final_dfs, ignore_index=True)
df_consolidado.shape

(3856577, 60)

# 03. Revisión de variables

In [10]:
df_unique = df_consolidado.nunique().sort_values(ascending = True)

In [11]:
# Revisamos variables con a lo más 10 valores únicos
aux_var = df_unique[df_unique <= 10].index.tolist()

In [12]:
for c in aux_var:    
    n_abs = df_consolidado[c].value_counts(dropna=False) # total de valores únicos
    n_rel = df_consolidado[c].value_counts(normalize=True, dropna=False).map('{:.2%}'.format) # Dist valores únicos
    
    # Resumen
    df_dist = pd.concat([n_abs, n_rel], axis=1)
    df_dist.columns = ['Conteo', 'Proporción']

    print(df_dist)

          Conteo Proporción
razon_m                    
0        3853192     99.91%
1           3385      0.09%
       Conteo Proporción
sexo                    
1     2209021     57.28%
2     1644558     42.64%
9        2998      0.08%
             Conteo Proporción
asist_medi                    
1           3108816     80.61%
2            495074     12.84%
9            252687      6.55%
            Conteo Proporción
necropsia                    
2          3023908     78.41%
9           491601     12.75%
1           341068      8.84%
             Conteo Proporción
nacionalid                    
1           3784199     98.12%
9             48673      1.26%
2             23705      0.61%
          Conteo Proporción
area_ur                    
1        3081939     79.91%
2         708277     18.37%
9          66361      1.72%
             Conteo Proporción
ocurr_trab                    
8           3526465     91.44%
2            161383      4.18%
9            141040      3.66%
1       

In [13]:
# Revisamos variables entre 10 y 40 valores únicos
aux_var2 = df_unique[(df_unique >= 10) & (df_unique <= 40)].index.tolist()

for c in aux_var2:    
    n_abs = df_consolidado[c].value_counts(dropna=False) # total de valores únicos
    n_rel = df_consolidado[c].value_counts(normalize=True, dropna=False).map('{:.2%}'.format) # Dist valores únicos
    
    # Resumen
    df_dist2 = pd.concat([n_abs, n_rel], axis=1)
    df_dist2.columns = ['Conteo', 'Proporción']

    print(df_dist2)

             Conteo Proporción
derechohab                    
2           1336074     34.64%
1           1311408     34.00%
99           569916     14.78%
3            276883      7.18%
7            173476      4.50%
8             99520      2.58%
4             30609      0.79%
9             26316      0.68%
5             21417      0.56%
6              6802      0.18%
10             4156      0.11%
             Conteo Proporción
lugar_ocur                    
88          3520158     91.28%
4            146673      3.80%
9             88132      2.29%
0             57658      1.50%
8             28958      0.75%
7              5727      0.15%
5              4721      0.12%
6              2218      0.06%
1              1121      0.03%
2               655      0.02%
3               556      0.01%
           Conteo Proporción
mes_regis                   
1          411164     10.66%
8          354623      9.20%
12         345566      8.96%
2          334591      8.68%
7          326907   

In [14]:
# Revisamos variables con más de 40 valores únicos
aux_var3 = df_unique[df_unique >= 40].index.tolist()

for c in aux_var3:    
    n_abs = df_consolidado[c].value_counts(dropna=False) # total de valores únicos
    n_rel = df_consolidado[c].value_counts(normalize=True, dropna=False).map('{:.2%}'.format) # Dist valores únicos
    
    # Resumen
    df_dist3 = pd.concat([n_abs, n_rel], axis=1)
    df_dist3.columns = ['Conteo', 'Proporción']

    print(df_dist3)

           Conteo Proporción
gr_lismex                   
 28        639236     16.58%
 20        563669     14.62%
 06        506868     13.14%
 33        316916      8.22%
 35        294781      7.64%
 30        144594      3.75%
E55        138012      3.58%
 27        121720      3.16%
 09        118851      3.08%
 38        104878      2.72%
 12         88285      2.29%
 29         70111      1.82%
E51         63690      1.65%
E49         62833      1.63%
 23         53843      1.40%
 11         49573      1.29%
 48         44784      1.16%
 46         40902      1.06%
 14         37286      0.97%
E54         33639      0.87%
 13         31850      0.83%
 47         31322      0.81%
 10         30359      0.79%
 21         27355      0.71%
 03         26845      0.70%
 18         26707      0.69%
 37         22014      0.57%
 19         21235      0.55%
 22         20782      0.54%
E56         20022      0.52%
 31         16910      0.44%
 36         16855      0.44%
 01         12

# 04. Eliminación de variables

In [15]:
drop_var = ['razon_m', 'nacionalid', 'ocurr_trab', 
            'anio_regis', 'mes_regis', 'dia_regis',
            'ent_regis', 'mun_regis',
            'rel_emba', 'complicaro', 'vio_fami', 'lengua',
            'anio_cert', 'mes_cert', 'dia_cert', 
            'embarazo', 'dis_re_oax', 'par_agre', 'maternas',
            'ent_ocules', 'mun_ocules', 'loc_ocules',
            'capitulo', 'lista1', 'gr_lismex', 'lista_mex']

In [16]:
len(drop_var)

26

In [17]:
valida_drop = list(set(drop_var).intersection(set(df_consolidado.columns)))
len(valida_drop)

26

In [18]:
df_consolidado.shape

(3856577, 60)

In [19]:
df_reducido = df_consolidado.drop(columns = valida_drop, inplace = False)
df_reducido.shape

(3856577, 34)

In [20]:
df_unique2 = df_reducido.nunique().sort_values(ascending = True)

In [21]:
df_unique2

sexo             3
area_ur          3
necropsia        3
asist_medi       3
cond_act         4
year_db          4
tipo_defun       7
cond_cert        7
edo_civil        8
derechohab      11
lugar_ocur      11
escolarida      12
sitio_ocur      13
mes_ocurr       13
mes_nacim       13
tloc_resid      18
tloc_ocurr      18
horas           25
edad_agru       30
dia_ocurr       32
dia_nacim       32
ent_ocurr       33
grupo           34
ent_resid       36
minutos         61
ocupacion       67
anio_ocur       81
anio_nacim     126
edad           188
mun_ocurr      571
mun_resid      571
loc_ocurr     1036
loc_resid     1206
causa_def     5404
dtype: int64

# 05. Salida

In [22]:
file_name = 'df_reducido.parquet'
output_path = os.path.join(base_path, 'data_procesada')
parquet_file = os.path.join(output_path, file_name)
df_reducido.to_parquet(parquet_file, index=False)